# Calculate biomass from deadwood

# Imports and Set-up

In [1]:
# Standard Imports
import sys
import pandas as pd
import numpy as np

# Google Cloud Imports
import pandas_gbq

In [2]:
# Util imports
sys.path.append("../../")  # include parent directory
from src.settings import (
    GCP_PROJ_ID,
    CARBON_POOLS_OUTDIR,
)

from src.biomass_equations import vmd0002_eq2, vmd0002_eq7, vmd0002_eq8

In [34]:
# Variables
PLOT_INFO_CSV = CARBON_POOLS_OUTDIR / "plot_info.csv"
STUMPS_CSV = CARBON_POOLS_OUTDIR / "stumps.csv"
LDW_CSV = CARBON_POOLS_OUTDIR / "lying_deadwood_wo_hollow.csv"
LDW_HOLLOW_CSV = CARBON_POOLS_OUTDIR / "lying_deadwood_hollow.csv"

# BigQuery Variables
DATASET_ID = "carbon_stock"
IF_EXISTS = "replace"

# Processing Conditions
OUTLIER_REMOVAL = "get_ave"  # Options: "get_ave", "drop_outliers", "eq_150"

## Load data

### Plot Data

In [4]:
if PLOT_INFO_CSV.exists():
    plot_info = pd.read_csv(PLOT_INFO_CSV)
else:
    query = f"""
    SELECT
        * 
    FROM {GCP_PROJ_ID}.{DATASET_ID}.plot_info"""

    # Read the BigQuery table into a dataframe
    plot_info = pandas_gbq.read_gbq(query, project_id=GCP_PROJ_ID)
    plot_info.to_csv(PLOT_INFO_CSV, index=False)

In [5]:
plot_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 674 entries, 0 to 673
Data columns (total 29 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   unique_id               674 non-null    object 
 1   data_recorder           674 non-null    object 
 2   team_no                 674 non-null    int64  
 3   plot_code_nmbr          674 non-null    int64  
 4   plot_type               674 non-null    object 
 5   sub_plot                674 non-null    object 
 6   yes_no                  674 non-null    object 
 7   sub_plot_shift          634 non-null    object 
 8   GPS_waypt               634 non-null    float64
 9   GPS_id                  634 non-null    float64
 10  GPS                     577 non-null    object 
 11  GPS_latitude            577 non-null    float64
 12  GPS_longitude           577 non-null    float64
 13  GPS_altitude            577 non-null    float64
 14  GPS_precision           577 non-null    fl

### Stumps

In [6]:
if STUMPS_CSV.exists():
    stumps = pd.read_csv(STUMPS_CSV)
else:
    query = f"""
    SELECT
        * 
    FROM {GCP_PROJ_ID}.{DATASET_ID}.stumps"""

    # Read the BigQuery table into a dataframe
    stumps = pandas_gbq.read_gbq(query, project_id=GCP_PROJ_ID)
    stumps.to_csv(STUMPS_CSV, index=False)

In [7]:
stumps.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1754 entries, 0 to 1753
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   unique_id      1754 non-null   object 
 1   nest           1754 non-null   int64  
 2   Diam1          1754 non-null   float64
 3   Diam2          1754 non-null   float64
 4   slope          1754 non-null   float64
 5   height         1754 non-null   float64
 6   cut_cl         1754 non-null   object 
 7   hollow_go      1754 non-null   object 
 8   hollow_d1      171 non-null    float64
 9   hollow_d2      171 non-null    float64
 10  stump_density  1754 non-null   float64
dtypes: float64(7), int64(1), object(3)
memory usage: 150.9+ KB


### Lying deadwood

In [35]:
if LDW_CSV.exists():
    ldw = pd.read_csv(LDW_CSV)
else:
    query = f"""
    SELECT
        * 
    FROM {GCP_PROJ_ID}.{DATASET_ID}.lying_deadwood_wo_hollow"""

    # Read the BigQuery table into a dataframe
    ldw = pandas_gbq.read_gbq(query, project_id=GCP_PROJ_ID)
    ldw.to_csv(LDW_CSV, index=False)

In [36]:
ldw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1826 entries, 0 to 1825
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   unique_id   1826 non-null   object 
 1   repetition  1826 non-null   int64  
 2   type        1826 non-null   object 
 3   class       1826 non-null   object 
 4   diameter    1826 non-null   float64
 5   density     1826 non-null   float64
dtypes: float64(2), int64(1), object(3)
memory usage: 85.7+ KB


In [37]:
if LDW_HOLLOW_CSV.exists():
    ldw_hollow = pd.read_csv(LDW_HOLLOW_CSV)
else:
    query = f"""
    SELECT
        * 
    FROM {GCP_PROJ_ID}.{DATASET_ID}.lying_deadwood_hollow"""

    # Read the BigQuery table into a dataframe
    ldw_hollow = pandas_gbq.read_gbq(query, project_id=GCP_PROJ_ID)
    ldw_hollow.to_csv(LDW_HOLLOW_CSV, index=False)

In [38]:
ldw_hollow.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   unique_id   15 non-null     object 
 1   repetition  15 non-null     int64  
 2   type        15 non-null     object 
 3   class       15 non-null     object 
 4   hollow_d1   15 non-null     float64
 5   hollow_d2   15 non-null     float64
 6   diameter    15 non-null     float64
 7   density     15 non-null     float64
dtypes: float64(4), int64(1), object(3)
memory usage: 1.1+ KB


# Calculate stump biomass

In [8]:
stumps.head(2)

,unique_id,nest,Diam1,Diam2,slope,height,cut_cl,hollow_go,hollow_d1,hollow_d2,stump_density
0,308C2,2,30.0,29.0,43.0,18.0,saw_axe,no,NaN,NaN,1.0
1,249B2,2,15.0,10.0,51.0,80.0,saw_axe,no,NaN,NaN,3.0


In [13]:
# get wood density equivalent for each density class

density_val = {1: 0.54, 2: 0.35, 3: 0.21}
stumps["stump_density_val"] = stumps["stump_density"].replace(density_val).fillna(0.21)

In [21]:
# Get biomass for each stump
stumps = vmd0002_eq2(stumps, "Diam1", "Diam2", "height", "stump_density_val")

In [22]:
# Get biomass of each stump that is hollow
stumps_hollow = vmd0002_eq2(
    stumps, "hollow_d1", "hollow_d2", "height", "stump_density_val"
)

In [25]:
# Get biomass to subtract due to hollow stumps
stumps["biomass_hollow"] = stumps_hollow["biomass"]

In [28]:
# Subtract biomass of hollow stumps from total biomass
stumps["biomass"] = np.where(
    (~stumps["biomass_hollow"].isna()) & (stumps["biomass_hollow"] > 0),
    stumps["biomass"] - stumps["biomass_hollow"],
    stumps["biomass"],
)

In [32]:
# Remove biomass_hollow column to avoid confusion
stumps.drop(columns=["biomass_hollow"], inplace=True)

In [33]:
stumps.head(2)

,unique_id,nest,Diam1,Diam2,slope,height,cut_cl,hollow_go,hollow_d1,hollow_d2,stump_density,stump_density_val,biomass
0,308C2,2,30.0,29.0,43.0,18.0,saw_axe,no,NaN,NaN,1.0,0.54,2.8674
1,249B2,2,15.0,10.0,51.0,80.0,saw_axe,no,NaN,NaN,3.0,0.21,2.1000


# Calculate Lying deadwood biomass

In [39]:
ldw.head(2)

,unique_id,repetition,type,class,diameter,density
0,308D2,1,tr1,FC,16.5,3.0
1,308D2,2,tr1,FC,18.3,3.0


## No hollow

### Outlier removal

In [40]:
ldw.describe()

,repetition,diameter,density
count,1826.000000,1826.000000,1826.000000
mean,2.529573,27.733866,1.937021
std,1.943715,109.834565,0.744572
min,1.000000,10.000000,1.000000
25%,1.000000,14.425000,1.000000
50%,2.000000,20.000000,2.000000
75%,3.000000,30.000000,2.000000
max,15.000000,4592.000000,3.000000


In [46]:
# Filter the ldw DataFrame to keep rows where diameter is less than or equal to the 98th percentile
ldw = ldw[ldw["diameter"] <= 150]

In [63]:
ldw = vmd0002_eq7(ldw, "diameter")

In [67]:
ldw = vmd0002_eq8(ldw, "density")

In [68]:
ldw.head(2)

,unique_id,repetition,type,class,diameter,density,deadwood_volume,biomass
0,308D2,1,tr1,FC,16.5,3.0,3.35875,0.705337
1,308D2,2,tr1,FC,18.3,3.0,4.13154,0.867623


## Hollow Lying Deadwood